In [ ]:
# # Import necessary modules
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import train_test_split

# # Create training and test sets
# X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.3, random_state=42)

# # Create the regressor: reg_all
# reg_all = LinearRegression()

# # Fit the regressor to the training data
# reg_all.fit(X_train,y_train)

# # Predict on the test data: y_pred
# y_pred = reg_all.predict(X_test)

# # Compute and print R^2 and RMSE
# print("R^2: {}".format(reg_all.score(X_test, y_test)))
# rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# print("Root Mean Squared Error: {}".format(rmse))


In [5]:
import pandas as pd
df = pd.read_csv('updated_cleaned_data.csv')
df.describe().round()


,Property ID,Zip code,Price (€),Construction year,Bedrooms,Living area (m²),Furnished,How many fireplaces?,Terrace,Terrace surface (m²),...,Surface of the plot (m²),Number of frontages,Swimming pool,Elevator,Basement,Primary energy consumption (kWh/m²),CO₂ emission (kg CO₂/m²),Kitchen type scale,Building condition scale,Price per m²
count,15146.0,15146.0,15146.0,9869.0,15146.0,15146.0,15146.0,15146.0,15146.0,15146.0,...,9278.0,11826.0,15146.0,15146.0,15146.0,12316.0,3533.0,10515.0,11537.0,15146.0
mean,10481706.0,4730.0,574772.0,1974.0,3.0,217.0,0.0,0.0,1.0,13.0,...,2731.0,3.0,0.0,0.0,0.0,365.0,964.0,2.0,4.0,2852.0
std,200625.0,3069.0,629446.0,41.0,2.0,208.0,0.0,0.0,0.0,33.0,...,29948.0,1.0,0.0,0.0,0.0,1686.0,4797.0,1.0,1.0,1953.0
min,4611027.0,1000.0,19000.0,1753.0,1.0,14.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,-10.0,-4.0,0.0,1.0,32.0
25%,10470283.0,1950.0,273611.0,1956.0,2.0,114.0,0.0,0.0,0.0,0.0,...,260.0,2.0,0.0,0.0,0.0,176.0,41.0,2.0,4.0,1839.0
50%,10561596.0,3970.0,395000.0,1978.0,3.0,170.0,0.0,0.0,1.0,0.0,...,659.0,3.0,0.0,0.0,0.0,272.0,67.0,2.0,4.0,2486.0
75%,10573953.0,8300.0,635000.0,2007.0,4.0,254.0,0.0,0.0,1.0,16.0,...,1448.0,4.0,0.0,0.0,1.0,406.0,111.0,3.0,6.0,3290.0
max,10584959.0,9992.0,11000000.0,2033.0,32.0,11000.0,1.0,6.0,1.0,1500.0,...,2400000.0,14.0,1.0,1.0,1.0,85624.0,88028.0,3.0,6.0,58824.0


In [6]:
df.isnull().sum()

Property ID                                0
Zip code                                   0
City                                       0
Type of Property                           0
Subtype of Property                        0
Price (€)                                  0
Construction year                       5277
Bedrooms                                   0
Living area (m²)                           0
Kitchen type                            4631
Furnished                                  0
How many fireplaces?                       0
Terrace                                    0
Terrace surface (m²)                       0
Garden                                     0
Garden surface (m²)                        0
Surface of the plot (m²)                5868
Number of frontages                     3320
Swimming pool                              0
Building condition                      3609
Elevator                                   0
Basement                                   0
Primary en

In [14]:
df.loc[df['Swimming pool'] == 0,'Swimming pool'].count()

14416

In [15]:
df['Swimming pool'].count()

15146

In [16]:
ratio = df.loc[df['Swimming pool'] == 0,'Swimming pool'].count() * 100/ df['Swimming pool'].count()
ratio

95.18024560940182